# Predictions View

Currently using PDC data to reproduce the view of Predictions from a data dump: Lift, CTR, Performance trend charts.

Will be generalized to support a direct export from the Prediction Studio datamart table.

In [ ]:
from pathlib import Path
import polars as pl

pdc_data_file = Path(
    # "~/<YOUR PATH TO THE PDC EXPORT>.parquet"
    "/Users/perdo/Library/CloudStorage/OneDrive-SharedLibraries-PegasystemsInc/AI Chapter Data Sets - Documents/Customers/Bank of New Zealand/PDC/bank-of-new-zealand.bofnz-bnzcee-prod1.telemetry.parquet"
).expanduser()
pdc_data = pl.read_parquet(pdc_data_file).filter(pl.col("ModelType").str.starts_with("Prediction")).sort("SnapshotTime")

pdc_data.head()

In [ ]:
pdc_data.filter(pl.col("SnapshotTime") == pl.col("SnapshotTime").last()).sort(["SnapshotTime", "ModelName", "ModelType"]).select(["ModelName", "ModelType", "Negatives", "Positives"]).head(50).to_pandas().style.hide()

In [ ]:
import plotly.express as px

px.line(
    pdc_data,
    x = "SnapshotTime",
    y = "Performance",
    color="ModelName")

In [ ]:
px.line(
    pdc_data,
    x = "SnapshotTime",
    y = "CTR",
    color="ModelName")

In [ ]:
from pdstools  import Prediction


p = Prediction(pdc_data.lazy())
p.summary_by_channel().collect().to_pandas().style

In [ ]:
px.line(
    p.summary_by_channel(keep_trend_data=True)
    .collect()
    .filter(pl.col("isMultiChannelPrediction").not_())
    .sort(["SnapshotTime"]),
    x="SnapshotTime",
    y="Performance",
    color="Channel",
)

In [ ]:
px.line(
    p.summary_by_channel(keep_trend_data=True)
    .collect()
    .filter(pl.col("isMultiChannelPrediction").not_())
    .sort(["SnapshotTime"]),
    x="SnapshotTime",
    y="Lift",
    color="Channel",
)